# ACME Customer Collaboration — Cedar + Amazon Verified Permissions

This notebook accompanies **Chapter 9** and the appendix of *Authorization in Digital Identity*.
It walks through creating an AVP policy store, uploading the Cedar schema and policies, defining entities,
and evaluating authorization requests with `is-authorized`.

The files that accompany this notebook are available at the [ACME Cedar demo Github repo](https://github.com/windley/acme-cedar-demo)

**Note:** This notebook is designed to be run in your own environment with the AWS CLI configured.
It uses shell (`bash`) cells that call `aws verifiedpermissions ...`. You’ll need:
- AWS CLI v2 (`aws --version`)
- An AWS account and region where **Amazon Verified Permissions** is available (e.g., `us-east-1`)
- `jq` for a couple of JSON wrapping steps

If you prefer, you can copy/paste these commands into your own terminal.

## A.1 Prerequisites

In [1]:
%%bash
# Check AWS CLI installation and region (adjust as needed)
aws --version || true
aws configure get region || aws configure set region us-east-1

aws-cli/2.28.11 Python/3.13.7 Darwin/24.6.0 source/arm64
us-east-1


## A.2 Create a policy store

We’ll create a new policy store in **STRICT** mode so AVP validates policy text against the schema.

In [2]:
%%bash
# Create a policy store in STRICT validation mode and capture the ID
STORE_ID=$(aws verifiedpermissions create-policy-store   --validation-settings "mode=STRICT"   --query policyStoreId --output text)
echo "STORE_ID=${STORE_ID}"

# (Optional) persist locally for later shell sessions
echo -n "${STORE_ID}" > STORE_ID.txt

STORE_ID=7fFVq1AWRV6wxpZeLFfGfz


## A.3 Upload the Cedar schema

AVP expects the Cedar schema in **JSON form** as a string value in the `cedarJson` field.
We’ll place the JSON representation into `acme.cedarschema.json`, then wrap it into `schema-definition.json`.

**Saved schema JSON** to `/mnt/data/acme.cedarschema.json`.

In [10]:
%%bash
# Wrap the schema JSON into the cedarJson field acceptable by AVP
# Requires `jq`; if you don't have jq, see the alternative Python cell below.
jq -Rs '{cedarJson: .}' acme.cedarschema.json > schema-definition.json

STORE_ID=`cat STORE_ID.txt`

# Upload to AVP
aws verifiedpermissions put-schema   --policy-store-id "${STORE_ID}"   --definition file://schema-definition.json

{
    "policyStoreId": "7fFVq1AWRV6wxpZeLFfGfz",
    "namespaces": [
        "ACME"
    ],
    "createdDate": "2025-08-21T16:46:19.527971+00:00",
    "lastUpdatedDate": "2025-08-21T16:46:19.527971+00:00"
}


## A.4 Uploading policies to AVP

We’ll create the **customer view**, **employee view**, **owner baseline**, **delegatable share**, and
**global unmanaged-device forbid** policies. Each Cedar policy is wrapped for `create-policy` using `jq -Rs`.

### A.4.1 View policies — Customers

In [13]:
%%bash
# Wrap and upload the customer view policy
jq -Rs --arg desc "Customer view policy"   '{static:{description:$desc,statement:.}}'   policy-customer-view.cedar > policy-customer-view.json

STORE_ID=`cat STORE_ID.txt`

aws verifiedpermissions create-policy   --policy-store-id "${STORE_ID}"   --definition file://policy-customer-view.json

{
    "policyStoreId": "7fFVq1AWRV6wxpZeLFfGfz",
    "policyId": "Bq1JmRWfdNrLmKWdPK67Lp",
    "policyType": "STATIC",
    "actions": [
        {
            "actionType": "ACME::Action",
            "actionId": "doc:view"
        }
    ],
    "createdDate": "2025-08-21T16:48:00.460161+00:00",
    "lastUpdatedDate": "2025-08-21T16:48:00.460161+00:00",
    "effect": "Permit"
}


### A.4.1 View policies — Employees

In [15]:
%%bash
jq -Rs --arg desc "Employee view policy"   '{static:{description:$desc,statement:.}}'   policy-employee-view.cedar > policy-employee-view.json

STORE_ID=`cat STORE_ID.txt`

aws verifiedpermissions create-policy   --policy-store-id "${STORE_ID}"   --definition file://policy-employee-view.json

{
    "policyStoreId": "7fFVq1AWRV6wxpZeLFfGfz",
    "policyId": "HyUnWSe8reMvMgLdyFMVkN",
    "policyType": "STATIC",
    "actions": [
        {
            "actionType": "ACME::Action",
            "actionId": "doc:view"
        }
    ],
    "createdDate": "2025-08-21T16:48:51.705168+00:00",
    "lastUpdatedDate": "2025-08-21T16:48:51.705168+00:00",
    "effect": "Permit"
}


### A.4.2 Owner can do everything

In [16]:
%%bash
jq -Rs --arg desc "Owner baseline: all actions on owned docs"   '{static:{description:$desc,statement:.}}'  policy-owner-all.cedar > policy-owner-all.json

STORE_ID=`cat STORE_ID.txt`

aws verifiedpermissions create-policy   --policy-store-id "${STORE_ID}"   --definition file://policy-owner-all.json

{
    "policyStoreId": "7fFVq1AWRV6wxpZeLFfGfz",
    "policyId": "77pVSZuJCfKjiQaGU1m8gp",
    "policyType": "STATIC",
    "actions": [
        {
            "actionType": "ACME::Action",
            "actionId": "doc:view"
        },
        {
            "actionType": "ACME::Action",
            "actionId": "doc:edit"
        },
        {
            "actionType": "ACME::Action",
            "actionId": "doc:share"
        }
    ],
    "createdDate": "2025-08-21T16:49:07.571740+00:00",
    "lastUpdatedDate": "2025-08-21T16:49:07.571740+00:00",
    "effect": "Permit"
}


### A.4.3 Employees can share if delegatable

In [17]:
%%bash
jq -Rs --arg desc "Employees can share if delegatable"   '{static:{description:$desc,statement:.}}'  policy-share.cedar > policy-share.json

STORE_ID=`cat STORE_ID.txt`

aws verifiedpermissions create-policy   --policy-store-id "${STORE_ID}"   --definition file://policy-share.json

{
    "policyStoreId": "7fFVq1AWRV6wxpZeLFfGfz",
    "policyId": "DqbhaEBDrVKzZHLt92X3jq",
    "policyType": "STATIC",
    "actions": [
        {
            "actionType": "ACME::Action",
            "actionId": "doc:share"
        }
    ],
    "createdDate": "2025-08-21T16:49:31.376160+00:00",
    "lastUpdatedDate": "2025-08-21T16:49:31.376160+00:00",
    "effect": "Permit"
}


### A.4.4 Forbid access from unmanaged devices

In [18]:
%%bash
jq -Rs --arg desc "Global forbid for unmanaged devices"   '{static:{description:$desc,statement:.}}'  policy-managed-device.cedar > policy-managed-device.json

STORE_ID=`cat STORE_ID.txt`

aws verifiedpermissions create-policy   --policy-store-id "${STORE_ID}"   --definition file://policy-managed-device.json

{
    "policyStoreId": "7fFVq1AWRV6wxpZeLFfGfz",
    "policyId": "QWB5iNYH11GX4Q9HRsPT8a",
    "policyType": "STATIC",
    "createdDate": "2025-08-21T16:49:58.422962+00:00",
    "lastUpdatedDate": "2025-08-21T16:49:58.422962+00:00",
    "effect": "Forbid"
}


## A.5 Defining entities

In production, ACME builds the entity snapshot from a **relationship graph** at request time.
For this walkthrough, we’ll keep a single file `acme-entities.json` in the AVP **entityList** format (memberships via `parents`).

**Saved entities** to `acme-entities.json`.

### A.5.5 Using the entity definitions

In this notebook, we re-use the same `acme-entities.json` for each request.
In ACME’s production system, the PEP would build a per-request slice from the relationship graph instead of loading a static file.

## A.6 Evaluating authorization requests

### A.6.1 Owner actions (Alice)

In [19]:
%%bash
STORE_ID=`cat STORE_ID.txt`

aws verifiedpermissions is-authorized   --policy-store-id "${STORE_ID}"   --principal entityType=ACME::Employee,entityId=alice   --action actionType=ACME::Action,actionId=doc:edit   --resource entityType=ACME::Document,entityId=q3-plan   --context '{"contextMap":{"device":{"record":{"managed":{"boolean":true}}}}}'   --entities file://acme-entities.json

{
    "decision": "ALLOW",
    "determiningPolicies": [
        {
            "policyId": "77pVSZuJCfKjiQaGU1m8gp"
        }
    ],
    "errors": []
}


### A.6.2 Customer viewing (Kate)

In [20]:
%%bash
STORE_ID=`cat STORE_ID.txt`

aws verifiedpermissions is-authorized   --policy-store-id "${STORE_ID}"   --principal entityType=ACME::Customer,entityId=kate   --action actionType=ACME::Action,actionId=doc:view   --resource entityType=ACME::Document,entityId=q3-plan   --entities file://acme-entities.json

{
    "decision": "ALLOW",
    "determiningPolicies": [
        {
            "policyId": "Bq1JmRWfdNrLmKWdPK67Lp"
        }
    ],
    "errors": []
}


### A.6.3 Employee sharing (Bob, managed device)

In [21]:
%%bash
STORE_ID=`cat STORE_ID.txt`

aws verifiedpermissions is-authorized   --policy-store-id "${STORE_ID}"   --principal entityType=ACME::Employee,entityId=bob   --action actionType=ACME::Action,actionId=doc:share   --resource entityType=ACME::Document,entityId=q3-plan   --context '{"contextMap":{"device":{"record":{"managed":{"boolean":true}}}}}'   --entities file://acme-entities.json

{
    "decision": "ALLOW",
    "determiningPolicies": [
        {
            "policyId": "DqbhaEBDrVKzZHLt92X3jq"
        }
    ],
    "errors": []
}


### A.6.4 Unmanaged devices (Bob → DENY)

In [22]:
%%bash
STORE_ID=`cat STORE_ID.txt`

aws verifiedpermissions is-authorized   --policy-store-id "${STORE_ID}"   --principal entityType=ACME::Employee,entityId=bob   --action actionType=ACME::Action,actionId=doc:view   --resource entityType=ACME::Document,entityId=q3-plan   --context '{"contextMap":{"device":{"record":{"managed":{"boolean":false}}}}}'   --entities file://acme-entities.json

{
    "decision": "DENY",
    "determiningPolicies": [
        {
            "policyId": "QWB5iNYH11GX4Q9HRsPT8a"
        }
    ],
    "errors": []
}


## A.7 Wrapping up the end-to-end example

You’ve seen how ACME’s Customer Collaboration platform uses Cedar with AVP to combine ownership baselines,
customer sharing, discretionary delegation, and global ABAC constraints. In production, replace the static entities file
with a per-request slice built by your PEP from the organization’s relationship graph, and keep policies under test to evolve safely.